<a href="https://colab.research.google.com/github/PeroronShine/AndroidApp/blob/main/ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
BASE_PATH = "/content/drive/MyDrive/cifar"
TRAIN_PATH = f"{BASE_PATH}/train/train"
TEST_PATH = f"{BASE_PATH}/test/output"
LABELS_PATH = f"{BASE_PATH}/trainLabels.csv"

# Загружаем метки
df_labels = pd.read_csv(LABELS_PATH)
le = LabelEncoder()
df_labels['label_encoded'] = le.fit_transform(df_labels['label'])
num_classes = len(le.classes_)

print(f"Количество классов: {num_classes}")
print(f"Примеры: \n{df_labels.head()}")

In [ ]:
from PIL import Image

def load_images_pil(image_dir, df, img_size=(224, 224)):
    all_files = set(os.listdir(image_dir))
    X = []
    y = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        fname = f"{row['id']}.png"
        if fname not in all_files:
            continue
        path = os.path.join(image_dir, fname)
        with Image.open(path) as img:
            img = img.convert('RGB')
            img = img.resize(img_size, Image.Resampling.LANCZOS)
            img_array = np.array(img, dtype=np.float32) / 255.0
        X.append(img_array)
        y.append(row['label_encoded'])

    return np.stack(X), np.array(y)

# Загружаем данные
X, y, train_filenames = load_images_pil(TRAIN_PATH, df_labels)
print(f"Размер X: {X.shape}")  # Должно быть (N, 224, 224, 3)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Замораживаем базовую сеть
base_model.trainable = False

# Добавляем свои слои
model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Компиляция
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7
)

history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [ ]:
plt.figure(figsize=(14, 5))

# Точность
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Точность по эпохам')
plt.xlabel('Эпоха')
plt.ylabel('Accuracy')
plt.legend()

# Потери
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Потери по эпохам')
plt.xlabel('Эпоха')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Выберем случайные 6 изображений из валидационного набора
indices = np.random.choice(range(len(X_val)), 6, replace=False)
plt.figure(figsize=(15, 10))

for i, idx in enumerate(indices):
    img = X_val[idx]
    true_label = le.classes_[y_val[idx]]

    pred_probs = model.predict(np.expand_dims(img, axis=0), verbose=0)
    pred_label = le.classes_[np.argmax(pred_probs)]

    plt.subplot(2, 3, i + 1)
    plt.imshow(img)
    plt.title(f"Истинно: {true_label}\nПредсказано: {pred_label}",
              color='green' if true_label == pred_label else 'red')
    plt.axis('off')

plt.suptitle("Примеры предсказаний модели", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
def load_test_images_for_resnet(image_dir, img_size=(224, 224)):
    X = []
    filenames = []

    all_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png'))]
    print(f"Найдено тестовых файлов: {len(all_files)}")

    for filename in tqdm(all_files, desc="Обработка теста"):
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size, interpolation=cv2.INTER_CUBIC)
        img = img.astype(np.float32) / 255.0

        X.append(img)
        filenames.append(filename)

    return np.array(X), filenames

X_test, test_filenames = load_test_images_for_resnet(TEST_PATH)

preds = model.predict(X_test)
y_test_pred_encoded = np.argmax(preds, axis=1)
y_test_pred = le.inverse_transform(y_test_pred_encoded)

In [ ]:
N = 6  # количество изображений для отображения

# Получим список всех тестовых файлов
test_image_paths = [os.path.join(TEST_PATH, f) for f in os.listdir(TEST_PATH) if f.lower().endswith('.png')]
np.random.shuffle(test_image_paths)
test_image_paths = test_image_paths[:N]

plt.figure(figsize=(15, 10))

for i, img_path in enumerate(test_image_paths):
    # Загрузка и обработка
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (224, 224))
    img_normalized = img_resized.astype(np.float32) / 255.0
    img_tensor = np.expand_dims(img_normalized, axis=0)

    # Предсказание
    pred_probs = model.predict(img_tensor, verbose=0)
    pred_label = le.classes_[np.argmax(pred_probs)]
    confidence = np.max(pred_probs)

    # Отображение
    plt.subplot(2, 3, i + 1)
    plt.imshow(img_rgb)
    plt.title(f"Предсказано: {pred_label}\n(доверие: {confidence:.2f})",
              color='blue', fontsize=10)
    plt.axis('off')

plt.suptitle("Примеры предсказаний на тестовых изображениях", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
test_results = pd.DataFrame({
    'id': [f.split('.')[0] for f in test_filenames],
    'label': y_test_pred
})

test_results.to_csv('/content/testLabels_ResNet50.csv', index=False)